In [1]:
import pandas as pd
from datetime import datetime
import json

import warnings
warnings.filterwarnings("ignore")

fase = '06_evaluar_prediccion'

In [2]:
with open('gen_config.json', 'r') as file:
    gen_config =json.load(file)

In [3]:
folder = gen_config['folder']

path_prod_data = gen_config['path_prod_data']
path_pred_test = gen_config['path_pred_test']
path_pred_futuro = gen_config['path_pred_futuro']

path_kaggle = gen_config['path_kaggle']

print(f"{'COMIENZA':-^100}")
print(f"{fase:-^100}")

----------------------------------------------COMIENZA----------------------------------------------
---------------------------------------06_evaluar_prediccion----------------------------------------


In [4]:
def perdida(real, prediccion):
    total = real.sum()
    perdida =  abs(real - prediccion).sum()
    resultado = perdida / total
    return resultado

In [5]:
prod_data = pd.read_parquet(f"{folder}/{path_prod_data}")
prod_a_predecir = pd.read_csv('../productos_a_predecir.txt', sep='\t')
df_test = pd.read_parquet(f"{folder}/{path_pred_test}")
df_fut = pd.read_parquet(f"{folder}/{path_pred_futuro}")

In [6]:
df_pred_test = df_test[['product_id', 'tn_futuro_real', 'tn_prediccion_real']].groupby('product_id', as_index=False).sum()
df_pred_test['error'] = -(df_pred_test['tn_futuro_real'] - df_pred_test['tn_prediccion_real'])/df_pred_test['tn_futuro_real']*100
df_pred_test['error_abs'] = -(df_pred_test['tn_futuro_real'] - df_pred_test['tn_prediccion_real'])
df_pred_test['weight'] = df_pred_test['tn_futuro_real']/df_pred_test['tn_futuro_real'].sum()
df_pred_test.sort_values('weight', ascending=False, inplace=True)

perdida = perdida(df_pred_test['tn_futuro_real'], df_pred_test['tn_prediccion_real'])
print(f"Perdida en test: {perdida}")
print(f"Primeras filas de test:\n{df_pred_test.iloc[:11,:4]}")

Perdida en test: 0.3302545206691455
Primeras filas de test:
    product_id  tn_futuro_real  tn_prediccion_real      error
0        20001      1504.68856         1276.902753 -15.138402
1        20002      1087.30855          946.916692 -12.911869
2        20003       892.50129          807.238412  -9.553250
3        20004       637.90002          640.712603   0.440913
4        20005       593.24443          613.617765   3.434223
31       20032       527.79811          650.756418  23.296466
8        20009       495.03574          466.081744  -5.848870
5        20006       417.23228          498.722791  19.531210
10       20011       392.38290          398.640481   1.594764
6        20007       390.43432          492.609652  26.169660
9        20010       359.59998          457.730497  27.288799


In [7]:
# df_pred_ungroup = df_pred_test[['group_index', 'tn_prediccion_real']]
# df_pred_ungroup = df_pred_ungroup.merge(prod_data_ungroup, on='group_index', how='left')
# df_pred_ungroup = df_pred_ungroup[['group_index', 'product_id', 'tn_prediccion_real', 'proporcion']]

# df_pred_ungroup = df_pred_ungroup[df_pred_ungroup['product_id'].isin(prod_a_predecir['product_id'])]
# df_pred_ungroup['prediccion_final'] = df_pred_ungroup['tn_prediccion_real'] * df_pred_ungroup['proporcion']
# df_pred_ungroup.sort_values('product_id', ascending=True, inplace=True)

# df_pred_ungroup[df_pred_ungroup['group_index'] == 281]

In [8]:
df_pred_fut = df_fut[['product_id', 'tn_futuro_real', 'tn_prediccion_real']].groupby('product_id', as_index=False).sum()
#df_pred_fut['error'] = -(df_pred_fut['tn_futuro_real'] - df_pred_fut['tn_prediccion_real'])/df_pred_fut['tn_futuro_real']*100
#df_pred_fut['error_abs'] = -(df_pred_fut['tn_futuro_real'] - df_pred_fut['tn_prediccion_real'])
#df_pred_fut['weight'] = df_pred_fut['tn_futuro_real']/df_pred_fut['tn_futuro_real'].sum()
#df_pred_fut.sort_values('weight', ascending=False, inplace=True)

print(f"Primeras filas de futuro:\n{df_pred_fut.iloc[:11,:4]}")

Primeras filas de futuro:
    product_id  tn_futuro_real  tn_prediccion_real
0        20001             0.0         1265.382363
1        20002             0.0         1050.836895
2        20003             0.0          828.640062
3        20004             0.0          648.268366
4        20005             0.0          601.896289
5        20006             0.0          515.520038
6        20007             0.0          494.198560
7        20008             0.0          449.627120
8        20009             0.0          494.872478
9        20010             0.0          468.568940
10       20011             0.0          400.418964


In [9]:
# df_fut_ungroup = df_fut[['group_index', 'tn_prediccion_real']]
# df_fut_ungroup = df_fut_ungroup.merge(prod_data_ungroup, on='group_index', how='left')
# df_fut_ungroup = df_fut_ungroup[['group_index', 'product_id', 'tn_prediccion_real', 'proporcion']]

# df_fut_ungroup = df_fut_ungroup[df_fut_ungroup['product_id'].isin(prod_a_predecir['product_id'])]
# df_fut_ungroup['prediccion_final'] = df_fut_ungroup['tn_prediccion_real'] * df_fut_ungroup['proporcion']
# df_fut_ungroup.sort_values('product_id', ascending=True, inplace=True)

In [10]:
prod_a_predecir = pd.read_csv('../productos_a_predecir.txt', sep='\t')

In [11]:
kaggle =  df_pred_fut[['product_id','tn_prediccion_real']]
kaggle['tn_prediccion_real'] = kaggle['tn_prediccion_real'].clip(lower=0)
kaggle.columns = ['product_id', 'tn']
kaggle = kaggle.merge(prod_a_predecir, on='product_id', how='right')
kaggle['tn'] = kaggle['tn'].fillna(0)

In [12]:
total = kaggle['tn'].sum()
#kaggle['tn'] = (kaggle['tn'] / total) * 30600
#correccion = total/30600
#kaggle['tn'] = kaggle['tn']/correccion
print(total)

30204.18734167254


In [13]:
kaggle.to_csv(f"{folder}/{datetime.now().strftime('%Y%m%d_%H%M%S')}_{path_kaggle}", index=False)

In [14]:
print(f"{fase:-^100}")
print(f"{'FINALIZA':-^100}\n\n\n")

---------------------------------------06_evaluar_prediccion----------------------------------------
----------------------------------------------FINALIZA----------------------------------------------



